In [ ]:
from commonfunctions import *
import math
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
img = read_img("/home/kamel/Desktop/Image/Project/kamel_trial/test.png")
bin_img = binarize_img (rgb2gray(img))
show_images([img,bin_img],["Original","Binarized"])

In [ ]:
import numpy as np
import cv2
import math
from scipy import ndimage


def get_edges(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_edges = cv2.Canny(img_gray, 100, 100, apertureSize=3)
    return img_edges
def rotate_img(img,img_edges):
    img_copy = np.copy(img)
    lines = cv2.HoughLinesP(img_edges, 1, math.pi / 180.0, 100, minLineLength=100, maxLineGap=5)
    angles = []
    for [[x1, y1, x2, y2]] in lines:
        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 0), 3)
        angle = math.degrees(math.atan2(y2 - y1, x2 - x1))
        angles.append(angle)
    median_angle = np.median(angles)
    img_rotated = ndimage.rotate(img_copy, median_angle)
    cv2.imwrite('fixed.jpg', img_rotated)  
    return img_rotated


def remove_lines(image):

    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # Remove horizontal
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20,1))
    detected_lines = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
    cnts = cv2.findContours(detected_lines, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        cv2.drawContours(image, [c], -1, (255,255,255), 2)

    # Repair image
    repair_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,5))
    result = 255 - cv2.morphologyEx(255 - image, cv2.MORPH_CLOSE, repair_kernel, iterations=1)

    cv2.imshow('thresh', thresh)
    cv2.imshow('detected_lines', detected_lines)
    cv2.imshow('image', image)
    cv2.imshow('result', result)
    cv2.waitKey()



img = cv2.imread('/home/kamel/Desktop/Image/Project/OMR_IP/input/test.png')
cv2.imwrite('Original.jpg', img)  
img_edges = get_edges(img)
rotated_img = rotate_img(img,img_edges)
remove_lines(rotated_img)